<a href="https://colab.research.google.com/github/iisuslik43/nlp/blob/master/hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install pymorphy2 >> /dev/null
! pip install transformers >> /dev/null
! pip install wandb >> /dev/null
! wandb login b569130067842c47dc059479b75930c8f598012a

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [0]:
import numpy as np
import torch
import torch.utils.data
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm_notebook as tqdm
import pickle
import torch.nn.functional as F
import os
import csv

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [0]:
from google.colab import drive
drive.mount('/content/drive')
! mkdir data
! cp "drive/My Drive/nlp/train_qa.csv" data

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
dataset = []
with open('data/train_qa.csv') as csvfile:
  spamreader = csv.reader(csvfile, delimiter='\t', quotechar='|')
  header = next(spamreader, None)
  for row in spamreader:
    row = row[0][1:-1].split('","')
    dataset.append(row[2:])

In [0]:
import re
count = 0
with_indexes = []
for text, q, a in dataset:
  a1 = a
  while a[-1] in '.?!':
    a = a[:-1]
  while a[0] in '.?!':
    a = a[1:]
  a = a.lstrip().rstrip()
  if a.lower() in text.lower():
    index = text.lower().index(a.lower())
    l = index
    r = l + len(a)
    with_indexes.append((text, a, q, [l, r]))

In [0]:
from transformers import BertTokenizer, BertModel
bert_tokenizer = BertTokenizer.from_pretrained(
    'bert-base-multilingual-cased',
    do_lower_case=False
)

In [0]:
def tokenize(text):
  return bert_tokenizer.tokenize(text)
def add_padding(tokens, max_len):
  n = len(tokens)
  pad_len = max_len - n
  mask = [1] * n + [0] * pad_len
  tokens += [bert_tokenizer.pad_token] * pad_len
  return tokens, mask

In [0]:
tokenized = []
for text, a, q, (l, r) in tqdm(with_indexes):
  text1, text2, text3 = tokenize(text[:l]), tokenize(text[l:r]), tokenize(text[r:])
  l, r = len(text1) + 1, len(text1) + 1 + len(text2)
  text = text1 + text2 + text3
  q = tokenize(q)
  all_text = [bert_tokenizer.cls_token] + text + [bert_tokenizer.sep_token] + q + [bert_tokenizer.sep_token]
  if len(all_text) > 512:
    continue
  tokenized.append((all_text, [l, r]))

In [0]:
text, a, q, _ = with_indexes[0]
all_text, (l, r) = tokenized[0]
a, all_text[l:r]

('в Древнем Египте', ['в', 'Д', '##рев', '##нем', 'Е', '##ги', '##п', '##те'])

In [0]:
max_text = np.max([len(text) for text, _ in tokenized])
max_text

In [0]:
texts = []
masks_text = []
a_indexes = []
for text, a_index in tqdm(tokenized):
  text, mask_text = add_padding(text, max_text)
  text = bert_tokenizer.convert_tokens_to_ids(text)
  texts.append(text)
  masks_text.append(mask_text)
  a_indexes.append(a_index)

In [0]:
class DataSet:
    def __init__(self, texts, masks_text, a_indexes):
        self.texts = torch.tensor(texts)
        self.masks_text = torch.tensor(masks_text)
        self.a_indexes = torch.tensor(a_indexes)
      
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.masks_text[idx], self.a_indexes[idx]

In [0]:
bert_model = BertModel.from_pretrained('bert-base-multilingual-cased')

In [0]:
class QAModel(nn.Module):
    def __init__(self, load_new=False):
        super(QAModel, self).__init__()
        if load_new:
          self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        else:
          self.bert = bert_model


        for param in self.bert.parameters():
            param.requires_grad = False

        layers = [
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(16, 2)
        ]
        self.layers = nn.Sequential(*layers)

    def forward(self, text, mask_text):
        # all text embedding is second value in returning tuple
        x = self.bert(text, attention_mask=mask_text)[1]
        x = self.layers(x)
        return x

In [0]:
BATCH_SIZE = 32
SPLIT = int(len(texts) * 0.8)
dataset_train = torch.utils.data.DataLoader(
    dataset=DataSet(texts[:SPLIT], masks_text[:SPLIT], a_indexes[:SPLIT]),
    batch_size=BATCH_SIZE,
    shuffle=True
)
dataset_test = torch.utils.data.DataLoader(
    dataset=DataSet(texts[SPLIT:], masks_text[SPLIT:], a_indexes[SPLIT:]),
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [0]:
criterion = torch.nn.MSELoss()
model = QAModel().to(device)
optimizer = torch.optim.Adam(params=model.parameters())

import wandb
wandb.init(project="hw4-qa")
wandb.watch(model)

In [0]:
n_epochs = 3
for epoch in tqdm(range(1, n_epochs + 1)):
    for x_batch, masks, indexes in tqdm(dataset_train):
        optimizer.zero_grad()
        x_batch, masks, indexes = x_batch.to(device), masks.to(device), indexes.float().to(device)
        #print(x_batch.shape, masks.shape, indexes.shape)
        output = model(x_batch, masks)
        #print(output.shape)
        loss = criterion(output, indexes)

        loss.backward()
        optimizer.step()

        train_loss = loss.item()

        with torch.no_grad():
            model.eval()
            for x_test, masks_test, indexes_test in dataset_test:
                x_test, masks_test, indexes_test = x_test.to(device), masks_test.to(device), indexes_test.float().to(device)
                output_test = model(x_test, masks_test)
                test_loss = criterion(output_test, indexes_test).item()
                break
            model.train()
        wandb.log({"Test Loss": test_loss,
                    "Train Loss": train_loss
                    })
model.save(f'model/model.pt')
model.save(os.path.join(wandb.run.dir, 'model.pt'))

In [0]:
with torch.no_grad():
  text, mask, a_index = torch.tensor(texts[:4]).to(device), torch.tensor(masks_text[:4]).to(device), torch.tensor(a_indexes[:4]).to(device)
  #print(text.shape, mask.shape, a_index.shape)
  res = model(text, mask)
res, a_index


In [0]:
dataset[:3]

[['Первые упоминания о строении человеческого тела встречаются в Древнем Египте. В XXVII веке до н. э. египетский врач Имхотеп описал некоторые органы и их функции, в частности головной мозг, деятельность сердца, распространение крови по сосудам. В древнекитайской книге Нейцзин (XI—VII вв. до н. э.) упоминаются сердце, печень, лёгкие и другие органы тела человека. В индийской книге Аюрведа ( Знание жизни , IX-III вв. до н. э.) содержится большой объём анатомических данных о мышцах, нервах, типах телосложения и темперамента, головном и спинном мозге.',
  'Где встречаются первые упоминания о строении человеческого тела?',
  'в Древнем Египте'],
 ['Телескоп имеет модульную структуру и содержит пять отсеков для оптических приборов. Один из отсеков в течение долгого времени (1993—2009 годы) занимала корректирующая оптическая система (COSTAR), установленная во время первой экспедиции обслуживания в 1993 году для компенсации неточности изготовления главного зеркала. Поскольку все приборы, уст

In [0]:
sentences = []
for text, q, a in dataset:
  sentences += text.split('.')

In [0]:
import pymorphy2
import re
analyzer = pymorphy2.MorphAnalyzer()
def tokenize(sentence):
  return [analyzer.parse(token.lower())[0].normal_form for token in re.split(r'[^\w]+', sentence) if token != '']

In [0]:
tokens = [tokenize(sentence) for sentence in tqdm(sentences)]

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_df = 0.8,
                        max_features=60_000, ngram_range=(1,3),
                        tokenizer = lambda x: x, lowercase=False)
tfidf.fit_transform(tokens)

In [0]:
with open('dataset_281937_1.txt') as f:
  for line in f:
    line = line.split('\t')
    _, q_id, text, q = line
    min_d = None
    min_s = None
    for sentence in text.split('.'):
      tokens_s = tokenize(sentence)
      v_s = tfidf.transform(tokens_s)
      
